<a href="https://colab.research.google.com/github/Patrick-Setubal/data_science_analytics/blob/main/alura_curso/formacao_data_science/CorretorOrtograficoNLP/CorretorOrtograficoNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criando Um Corretor Ortografico NLP

# Importações

In [27]:
# Importações
import requests
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Coletar e Limpar Dados

In [29]:
# Criar Função que separa as palavras dos tokens
def _separar_palavras(list_tokens):
  lista_palavras = []
  for token in list_tokens:
    if token.isalpha():
      lista_palavras.append(token)
  return lista_palavras

# Criar função para normalçizar a lista de palavras (tudo Minusculo)
def _normalizacao(lista_palavras):
  lista_normalizada = []
  for palavra in lista_palavras:
    lista_normalizada.append(palavra.lower())
  return lista_normalizada

# Função Principal para fazer a leitura dos dados
def ler_dados(link):
  dados = requests.get(link).text

  # Criando uma lista de tokens (ex:  "Hoje?" = ['Hoje', '?'] )
  list_tokens = nltk.tokenize.word_tokenize(dados)

  # Usando Funções
  lista_palavras = _separar_palavras(list_tokens)
  lista_normalizada = _normalizacao(lista_palavras)
  print(f"Possui {len(lista_normalizada)} palavras")
  return lista_normalizada


link = "https://raw.githubusercontent.com/Patrick-Setubal/data_science_analytics/main/alura_curso/formacao_data_science/CorretorOrtograficoNLP/artigos.txt"
lista_normalizada = ler_dados(link)


Possui 403104 palavras


# Funções Do corretor

In [30]:
# Variavel Global com todas as letras
letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'

# Insere letra entre as fatias
def _insere_letras(fatias):
  novas_palavras = []
  for E, D in fatias:
    for letra in letras:
      novas_palavras.append(E + letra + D)
  return novas_palavras

# Troca letra entre as fatias
def _troca_letras(fatias):
  novas_palavras = []
  for E, D in fatias:
    for letra in letras:
      novas_palavras.append(E + letra + D[1:])
  return novas_palavras

# Deleta letra entre as fatias
def _deletando_caracteres(fatias):
  novas_palavras = []
  for E, D in fatias:
    novas_palavras.append(E + D[1:])
  return novas_palavras


# Troca letra entre as fatias
def _inverte_letra(fatias):
  novas_palavras = []
  for E, D in fatias:
    if len(D)>1:
      novas_palavras.append(E + D[1] + D[0] + D[2:])
  return novas_palavras


# Gerar palavras 
def gerador_palavra(palavra):
  fatias = []
  for i in range(len(palavra)+1):
    fatias.append((palavra[:i],palavra[i:]))
    palavras_geradas = _insere_letras(fatias)
    palavras_geradas += _deletando_caracteres(fatias)
    palavras_geradas += _troca_letras(fatias)
    palavras_geradas += _inverte_letra(fatias)

  return palavras_geradas

# Define a probabilidade da palavra estar certa com base no dataset
# Variavel Global Utilizando a lista_normalizada
frequencia = nltk.FreqDist(lista_normalizada)
total_palavras = len(lista_normalizada)
def _probabilidade(palavra_gerada):
  return frequencia[palavra_gerada]/total_palavras

# Função principal Corrige a palavra
def corretor(palavra):
  palavras_geradas = gerador_palavra(palavra)
  palavra_correta = max(palavras_geradas, key=_probabilidade)
  return palavra_correta

# Funções do Avaliador

In [31]:
# coleta e trata os dados para teste
def cria_dados_teste(link_arquivo):
  lista_palavras_teste = []
  f = requests.get(link_arquivo).text.split('\n')
  for linha in f:
    if ' ' in linha:
      correta, errada = linha.split(' ')
      lista_palavras_teste.append((correta, errada))
  return lista_palavras_teste


# Função avaliador printa a porcentagem de acertos 
def avaliador(testes, vocabulario):
  numero_palavras = len(testes)
  acertou = 0
  desconhecida = 0
  for correta, errada in testes:
    desconhecida += (correta not in vocabulario)
    if correta == corretor(errada):
      acertou += 1
        


  taxa_desconhecida =  round(desconhecida*100/numero_palavras, 2)
  taxa_acerto = round(acertou*100/numero_palavras, 2)
  print(f'acertou {acertou} de {numero_palavras}')
  print(f'taxas acerto e de {taxa_acerto}%')
  print(f'taxas de palavra desconhecida de {taxa_desconhecida}%')


# Criar Dados Para Teste
link = 'https://raw.githubusercontent.com/Patrick-Setubal/data_science_analytics/main/alura_curso/formacao_data_science/CorretorOrtograficoNLP/palavras.txt'
lista_teste = cria_dados_teste(link)

avaliador(lista_teste,set(lista_normalizada))


acertou 142 de 186
taxas acerto e de 76.34%
taxas de palavra desconhecida de 6.99%


# Uso do Corretor


In [44]:
# lista de palavras a serem corregidas 
Uso_Corretor = ['logica', 'lgica', 'lógca', 'lógiac']
Uso_Corretor2 = ['achine', 'amchine','machiine','macoine',]
Uso_Corretor3 = ['learing', 'laerning','kearning','leearning',]

for palavra in Uso_Corretor:
  print(f'{palavra} = {corretor(palavra)}')

print('-------')

for palavra in Uso_Corretor2:
  print(f'{palavra} = {corretor(palavra)}')

print('-------')

for palavra in Uso_Corretor3:
  print(f'{palavra} = {corretor(palavra)}')

logica = lógica
lgica = lógica
lógca = lógica
lógiac = lógica
-------
achine = machine
amchine = machine
machiine = machine
macoine = machine
-------
learing = learning
laerning = learning
kearning = learning
leearning = learning
